In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Viz
import matplotlib.pyplot as plt
import seaborn as sns
 
# settings
import warnings
warnings.filterwarnings("ignore")
 
from  sklearn.ensemble import RandomForestClassifier
from  sklearn.ensemble import RandomForestRegressor

导入数据并查看数据

In [ ]:
train = pd.read_csv('../input/GiveMeSomeCredit/cs-training.csv')
test = pd.read_csv('../input/GiveMeSomeCredit/cs-test.csv')
data = pd.read_csv('../input/GiveMeSomeCredit/cs-training.csv')
p=0
for ff in train['SeriousDlqin2yrs']:
    if ff == 1:
        p=p+1
print(p)
print(len(test['SeriousDlqin2yrs']))
rr = p/len(test['SeriousDlqin2yrs'])
print(rr)
train

In [ ]:
test.info()

根据业务的背景，可以上客户信息分为以下几类：

基本信息：age

负债信息:RevolvingUtilizationOfUnsecuredLines、DebtRatio、NumberOfOpenCreditLinesAndLoans、NumberRealEstateLoansOrLines

偿还能力：MonthlyIncome

历史信用记录：NumberOfTime30-59DaysPastDueNotWorse、NumberOfTime60-89DaysPastDueNotWorse、NumberOfTimes90DaysLate

人际社交信息：NumberOfDependents

数据处理

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

#导入统计学常用数据包
from scipy import stats
from scipy.stats import norm, skew 

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #忽略警告
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #设置pandas输出数据为3位小数

重复值处理

In [ ]:
#保存ID号码
train_ID = train['Unnamed: 0']
test_ID = test['Unnamed: 0']
y_train = train.SeriousDlqin2yrs
y_test = test.SeriousDlqin2yrs
all_data = pd.concat((train, test)).reset_index(drop=True)

#检查数据
print("处理前数据大小 : {} ".format(all_data.shape))

#获取去重复的字段名列表
cols_df = all_data.columns.values.tolist()
cols_df.remove('Unnamed: 0')

#去除训练数据中的重复值
all_data.drop_duplicates(subset = cols_df, inplace=True)


#检查数据
print("处理后数据大小 : {} ".format(all_data.shape))

缺失值处理

In [ ]:
import missingno as msno
msno.matrix(all_data)

In [ ]:
all_data.isnull().mean()

对于numberofdependents缺失率较低，直接删除，对monthlyincome缺失率较高，后面根据分布特征选择缺失值填补方法

In [ ]:
#检查数据
print("处理前数据集大小 : {} ".format(all_data.shape))


all_data.dropna(subset=['NumberOfDependents'],inplace=True)

#检查数据
print('--------------------------------')
print("处理后数据集大小 : {} ".format(all_data.shape))

In [ ]:
# 本次单变量探索用到的字定义函数
##分位法处理异常值
def IQR_Cleaner(df,atrribute_name,floor,ceiling,method):
    ##df：要清洗的DataFrame，DataFrame
    ##atrribute_name：清洗的属性名称，字符串
    ##floor：清洗的下分位数，float
    ##ceiling：清洗的上分位数，float
    ##method:'drop' 或者 ‘fill’
    Q1 = df[atrribute_name].quantile(floor)
    Q3 = df[atrribute_name].quantile(ceiling)
    IQR = Q3 - Q1
    down = Q1 - 1.5*IQR
    up = Q3 + 1.5*IQR
    if method == 'drop':
        df = df.loc[(df[atrribute_name]<up) & (df[atrribute_name]>down),:]
    else:
        df.loc[df[atrribute_name]>up,atrribute_name]=up
        df.loc[df[atrribute_name]<down,atrribute_name]=down
    return df

##单变量分布可视化函数
def Distribution_worker(target,bins):
    ## target 需要探索的单变量
    ## 直方图
    fig = plt.figure(figsize=(15,5))
    plt.subplot(121)
    sns.distplot(target, fit=norm, bins=bins)
    (mu, sigma) = norm.fit(target)
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
    plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],loc='best')
    plt.ylabel('Frequency')
    plt.title('distribution')
    
    ##箱线图
    plt.subplot(122)
    sns.boxplot(y =target , width=0.2)

age分析

In [ ]:
#输入目标变量
target = all_data.age

#变量分布特征
Distribution_worker(target,30)

In [ ]:
#异常值处理
all_data = IQR_Cleaner(all_data, 'age', 0.25, 0.75, 'fill')

#输入目标变量
target = all_data.age

#处理后变量分布特征
Distribution_worker(target,30)
# 年龄等于0的异常值进行剔除
data = data[data['age']> 0]

revolvingutilizationofunsecuredlines分析

In [ ]:
#输入目标变量
target = all_data.RevolvingUtilizationOfUnsecuredLines

#变量分布特征
Distribution_worker(target,30)

In [ ]:
#异常值处理
outlier_num  = all_data.loc[all_data.RevolvingUtilizationOfUnsecuredLines>1].shape[0]
print(" RevolvingUtilizationOfUnsecuredLines字段大于1的记录占比: {} ".format(outlier_num/119587)) 

#大于1的记录赋值为1
all_data = all_data.loc[all_data.RevolvingUtilizationOfUnsecuredLines<=1]

#剔除异常值
data = data[data['RevolvingUtilizationOfUnsecuredLines'] < 6]

#输入目标变量
target = all_data.RevolvingUtilizationOfUnsecuredLines

#变量分布特征
Distribution_worker(target,30)

debtratio分析

In [ ]:
#输入目标变量
target = all_data.DebtRatio

#变量分布特征
Distribution_worker(target,30)
datatemp2=data["DebtRatio"]

In [ ]:
#异常值处理
outlier_num  = all_data.loc[all_data.DebtRatio>1].shape[0]
print(" DebtRatio字段大于1的记录占比: {} ".format(outlier_num/119587)) 

#只有几百个样本，对于15万数据来说不影响总体数据结构，所以剔除
data = data[data['DebtRatio'] < 8000]

#天花板法处理异常值
all_data = IQR_Cleaner(all_data, 'DebtRatio', 0.25, 0.75, 'fill')

#处理后字段分布特征

#输入目标变量
target = all_data.DebtRatio

#变量分布特征
Distribution_worker(target,30)

numberrealestateloansorlines分析

In [ ]:
#输入目标变量
target = all_data.NumberRealEstateLoansOrLines

#变量分布特征
Distribution_worker(target,5)

In [ ]:
#天花板法处理异常值
all_data = IQR_Cleaner(all_data, 'NumberRealEstateLoansOrLines', 0.25, 0.75, 'fill')

#剔除异常值
data = data[data['NumberRealEstateLoansOrLines'] < 30]

#输入目标变量
target = all_data.NumberRealEstateLoansOrLines

#变量分布特征
Distribution_worker(target,5)

numberofdependents字段

In [ ]:
#输入目标变量
target = all_data.NumberOfDependents

#变量分布特征
Distribution_worker(target,5)

In [ ]:
#天花板法处理异常值
all_data = IQR_Cleaner(all_data, 'NumberOfDependents', 0.25, 0.75, 'fill')

#剔除异常值
data = data[data['NumberOfTime30-59DaysPastDueNotWorse'] < 20]

#输入目标变量
target = all_data.NumberOfDependents

#变量分布特征
Distribution_worker(target,5)

历史信用记录分析

In [ ]:
##观察历史信用记录数据分布情况
target = all_data[['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']]
sns.boxplot(data =target, width=0.2,orient='h')

In [ ]:
#大于20的值赋值为20
all_data.loc[all_data['NumberOfTime30-59DaysPastDueNotWorse']>20,'NumberOfTime30-59DaysPastDueNotWorse']=20
all_data.loc[all_data['NumberOfTime60-89DaysPastDueNotWorse']>20,'NumberOfTime60-89DaysPastDueNotWorse']=20
all_data.loc[all_data['NumberOfTimes90DaysLate']>20,'NumberOfTimes90DaysLate']=20
##观察历史信用记录数据分布情况
target = all_data[['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']]
sns.boxplot(data =target, width=0.2,orient='h')

monthlyincome分析

In [ ]:
#输入目标变量
target = all_data.dropna().MonthlyIncome

#变量分布特征
Distribution_worker(target,30)

In [ ]:
#天花板法处理异常值
all_data = IQR_Cleaner(all_data, 'MonthlyIncome', 0.25, 0.75, 'fill')

#处理后字段分布特征
#剔除异常值
data = data[data['MonthlyIncome'] < 50000]

#输入目标变量
target = all_data.dropna().MonthlyIncome

#变量分布特征
Distribution_worker(target,30)

分箱处理

 变量分箱（binning）是对连续变量离散化（discretization）的一种称呼。

 信用评分卡开发中一般有常用的等距分段、等深分段、最优分段。

 其中等距分段（Equval length intervals）是指分段的区间是一致的，比如年龄以十年作为一个分段；

 等深分段（Equal frequency intervals）是先确定分段数量，然后令每个分段中数据数量大致相等；

 最优分段（Optimal Binning）又叫监督离散化（supervised discretizaion），使用递归划分（Recursive Partitioning）将连续变量分为分段，背后是一种基于条件推断查找较佳分组的算法。

 

 我们首先选择对连续变量进行最优分段，在连续变量的分布不满足最优分段的要求时，再考虑对连续变量进行等距分段。最优分箱的代码如下：

In [ ]:

from sklearn.model_selection import train_test_split
#data=data.drop(["Unnamed: 0"],axis=1)
Y = data['SeriousDlqin2yrs']
X = data.iloc[:, 1:]
#测试集占比30%
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
train = pd.concat([Y_train, X_train], axis=1)
test = pd.concat([Y_test, X_test], axis=1)
test.to_csv('TestData.csv',index=False)


# 定义自动分箱函数
 
from scipy import stats
def mono_bin(Y, X, n = 20):
    r = 0
    good=Y.sum()
    bad=Y.count()-good
    while np.abs(r) < 1:
        d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n,duplicates="drop")}) 
        # 后面报错You can drop duplicate edges by setting the 'duplicates' kwarg，所以回到这里补充duplicates参数
        # pandas中使用qcut()，边界易出现重复值，如果为了删除重复值设置 duplicates=‘drop’，则易出现于分片个数少于指定个数的问题
        # 经尝试，设置duplicates参数为“drop”可行，而不能设置为“raise”。
        d2 = d1.groupby('Bucket', as_index = True)
        r, p = stats.spearmanr(d2.X.mean(), d2.mean().Y)
        n = n - 1
    d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
    d3['min'] = d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe']=np.log((d3['rate']/(1-d3['rate']))/(good/bad))
    d4 = (d3.sort_values(by='min')).reset_index(drop=True)
    print("=" * 60)
    woe=list(d4['woe'].round(3))
    print(d4)
    return d4


变量分箱

In [ ]:

mono_bin(data.SeriousDlqin2yrs,data['DebtRatio'])
mono_bin(data.SeriousDlqin2yrs,data.RevolvingUtilizationOfUnsecuredLines)
mono_bin(data.SeriousDlqin2yrs,data.age)
mono_bin(data.SeriousDlqin2yrs,data.MonthlyIncome)
mono_bin(data.SeriousDlqin2yrs,data['NumberOfTime30-59DaysPastDueNotWorse'])
mono_bin(data.SeriousDlqin2yrs,data['NumberOfTime60-89DaysPastDueNotWorse'])
mono_bin(data.SeriousDlqin2yrs,data['NumberOfTimes90DaysLate'])
mono_bin(data.SeriousDlqin2yrs,data.NumberRealEstateLoansOrLines)
mono_bin(data.SeriousDlqin2yrs,data.NumberOfDependents)
mono_bin(data.SeriousDlqin2yrs,data.NumberOfOpenCreditLinesAndLoans)


NumberOfOpenCreditLinesAndLoans、NumberOfTimes90DaysLate、NumberOfTime60-89DaysPastDueNotWorse、NumberRealEstateLoansOrLines最终结果只分得一个组，那么针对不能最优分箱的变量，下面将进行自定义分箱。

In [ ]:
# 连续变量离散化
pinf = float('inf')#正无穷大
ninf = float('-inf')#负无穷大
 
cutx6 = [ninf, 1, 2, 3, 5, pinf]
cutx7 = [ninf, 0, 1, 3, 5, pinf]
cutx8 = [ninf, 0,1,2, 3, pinf]
cutx9 = [ninf, 0, 1, 3, pinf]

WoE分析， 是对指标分箱、计算各个档位的WoE值并观察WoE值随指标变化的趋势。其中WoE的数学定义是:
woe=ln(goodattribute/badattribute)
在进行分析时，我们需要对各指标从小到大排列，并计算出相应分档的WoE值。其中正向指标越大，WoE值越小；反向指标越大，WoE值越大。正向指标的WoE值负斜率越大，反响指标的正斜率越大，则说明指标区分能力好。WoE值趋近于直线，则意味指标判断能力较弱。若正向指标和WoE正相关趋势、反向指标同WoE出现负相关趋势，则说明此指标不符合经济意义，则应当予以去除。
woe函数实现在上面的mono_bin()函数里面已经包含，这里不再重复。
接下来，我们会用经过清洗后的数据看一下变量间的相关性。注意，这里的相关性分析只是初步的检查，进一步检查模型的IV（证据权重）作为变量筛选的依据

In [ ]:

corr = all_data.drop(['Unnamed: 0','SeriousDlqin2yrs'],axis=1).corr()
print(max(corr))
plt.subplots(figsize=(12, 12))
sns.heatmap(corr, annot=True, vmax=1, square=True, cmap='Blues')
corr = data.corr()#计算各变量的相关性系数




接下来，我进一步计算每个变量的Infomation Value（IV）。IV指标是一般用来确定自变量的预测能力。 其公式为：
IV=sum((goodattribute-badattribute)*ln(goodattribute/badattribute))
通过IV值判断变量预测能力的标准是：
 < 0.02: unpredictive
0.02 to 0.1: weak
0.1 to 0.3: medium
0.3 to 0.5: strong
> 0.5: suspicious

IV的实现放在mono_bin()函数里面，代码实现如下


In [ ]:
# IV的实现放在mono_bin()函数里面，代码实现如下：
 
# 定义自动分箱函数
def mono_bin(Y, X, n = 20):
    r = 0
    good=Y.sum()
    bad=Y.count()-good
    while np.abs(r) < 1:
        d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n,duplicates="drop")})
        d2 = d1.groupby('Bucket', as_index = True)
        r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
        n = n - 1
    d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
    d3['min']=d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe']=np.log((d3['rate']/(1-d3['rate']))/(good/bad))
    d3['goodattribute']=d3['sum']/good
    d3['badattribute']=(d3['total']-d3['sum'])/bad
    iv=((d3['goodattribute']-d3['badattribute'])*d3['woe']).sum()
    d4 = (d3.sort_values(by = 'min')).reset_index(drop=True)
    print("=" * 60)
    print(d4)
    cut=[]
    cut.append(float('-inf'))
    for i in range(1,n+1):
        qua=X.quantile(i/(n+1))
        cut.append(round(qua,4))
    cut.append(float('inf'))
    woe=list(d4['woe'].round(3))
    return d4,iv,cut,woe
 
#自定义分箱函数
# ——该定义函数参考另一篇帖子：https://blog.csdn.net/sunyaowu315/article/details/82981216 
def self_bin(Y,X,cat):
    good=Y.sum()
    bad=Y.count()-good
    d1=pd.DataFrame({'X':X,'Y':Y,'Bucket':pd.cut(X,cat)})
    d2=d1.groupby('Bucket', as_index = True)
    d3 = pd.DataFrame(d2.X.min(), columns=['min'])
    d3['min'] = d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe'] = np.log((d3['rate'] / (1 - d3['rate'])) / (good / bad))
    d3['goodattribute'] = d3['sum'] / good
    d3['badattribute'] = (d3['total'] - d3['sum']) / bad
    iv = ((d3['goodattribute'] - d3['badattribute']) * d3['woe']).sum()
    d4 = (d3.sort_values(by='min'))
    print("=" * 60)
    print(d4)
    woe = list(d4['woe'].round(3))
    return d4, iv,woe

In [ ]:
dfx1, ivx1,cutx1,woex1 = mono_bin(data.SeriousDlqin2yrs, data.RevolvingUtilizationOfUnsecuredLines,n=10)
dfx2, ivx2,cutx2,woex2 = mono_bin(data.SeriousDlqin2yrs, data.age, n=10)
dfx4, ivx4,cutx4,woex4 =mono_bin(data.SeriousDlqin2yrs, data.DebtRatio, n=20)
dfx5, ivx5,cutx5,woex5 =mono_bin(data.SeriousDlqin2yrs, data.MonthlyIncome, n=10)
 
# 连续变量离散化
cutx3 = [ninf, 0, 1, 3, 5, pinf]
cutx6 = [ninf, 1, 2, 3, 5, pinf]
cutx7 = [ninf, 0, 1, 3, 5, pinf]
cutx8 = [ninf, 0,1,2, 3, pinf]
cutx9 = [ninf, 0, 1, 3, pinf]
cutx10 = [ninf, 0, 1, 2, 3, 5, pinf]
dfx3, ivx3,woex3 = self_bin(data.SeriousDlqin2yrs, data['NumberOfTime30-59DaysPastDueNotWorse'], cutx3)
dfx6, ivx6,woex6= self_bin(data.SeriousDlqin2yrs, data['NumberOfOpenCreditLinesAndLoans'], cutx6)
dfx7, ivx7,woex7 = self_bin(data.SeriousDlqin2yrs, data['NumberOfTimes90DaysLate'], cutx7)
dfx8, ivx8,woex8 = self_bin(data.SeriousDlqin2yrs, data['NumberRealEstateLoansOrLines'], cutx8)
dfx9, ivx9,woex9 = self_bin(data.SeriousDlqin2yrs, data['NumberOfTime60-89DaysPastDueNotWorse'], cutx9)
dfx10, ivx10,woex10 = self_bin(data.SeriousDlqin2yrs, data['NumberOfDependents'], cutx10)

In [ ]:
# 生成的IV图代码：
 
ivlist=[ivx1,ivx2,ivx3,ivx4,ivx5,ivx6,ivx7,ivx8,ivx9,ivx10]#各变量IV
index=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']#x轴的标签
fig1 = plt.figure(1)
ax1 = fig1.add_subplot(1, 1, 1)
x = np.arange(len(index))+1
ax1.bar(x, ivlist, width=0.4)#生成柱状图
ax1.set_xticks(x)
ax1.set_xticklabels(index, rotation=0, fontsize=12)
ax1.set_ylabel('IV(Information Value)', fontsize=14)
#在柱状图上添加数字标签
for a, b in zip(x, ivlist):
    plt.text(a, b + 0.01, '%.4f' % b, ha='center', va='bottom', fontsize=10)
plt.show()

证据权重（Weight of Evidence,WOE）转换可以将Logistic回归模型转变为标准评分卡格式。引入WOE转换的目的并不是为了提高模型质量，只是一些变量不应该被纳入模型，这或者是因为它们不能增加模型值，或者是因为与其模型相关系数有关的误差较大，其实建立标准信用评分卡也可以不采用WOE转换。这种情况下，Logistic回归模型需要处理更大数量的自变量。尽管这样会增加建模程序的复杂性，但最终得到的评分卡都是一样的。
在建立模型之前，我们需要将筛选后的变量转换为WoE值，便于信用评分。

WOE转换

我们已经能获取了每个变量的分箱数据和woe数据，只需要根据各变量数据进行替换，实现代码如下：


In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn import metrics
import statsmodels.api as sm
import math
 
def trans_woe(var,var_name,x_woe,x_cut):
    woe_name = var_name + '_woe'
    for i in range(len(x_woe)):
        if i == 0:
            var.loc[(var[var_name]<=x_cut[i+1]),woe_name] = x_woe[i]
        elif (i>0) and (i<= len(x_woe)-2):
            var.loc[((var[var_name]>x_cut[i])&(var[var_name]<=x_cut[i+1])),woe_name] = x_woe[i]
        else:
            var.loc[(var[var_name]>x_cut[len(x_woe)-1]),woe_name] = x_woe[len(x_woe)-1]
    return var
 
x1_name = 'RevolvingUtilizationOfUnsecuredLines'
x2_name = 'age'
x3_name = 'NumberOfTime30-59DaysPastDueNotWorse'
x7_name = 'NumberOfTimes90DaysLate'
x9_name = 'NumberOfTime60-89DaysPastDueNotWorse'
 
 
X_train = trans_woe(X_train,x1_name,woex1,cutx1)
X_train = trans_woe(X_train,x2_name,woex2,cutx2)
X_train = trans_woe(X_train,x3_name,woex3,cutx3)
X_train = trans_woe(X_train,x7_name,woex7,cutx7)
X_train = trans_woe(X_train,x9_name,woex9,cutx9)

Logisic模型建立和模型检验

我们直接调用statsmodels包来实现逻辑回归：

In [ ]:




X_train.to_csv('WoeData.csv', index=False)
#6.2 Logistic模型建立
#导入数据
data = pd.read_csv('WoeData.csv')




In [ ]:
from sklearn.model_selection import train_test_split
#data=data.drop(["Unnamed: 0"],axis=1)
Y = data['SeriousDlqin2yrs']
X = data.iloc[:, 1:]
#测试集占比30%
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
train = pd.concat([Y_train, X_train], axis=1)
test = pd.concat([Y_test, X_test], axis=1)
test.to_csv('TestData.csv',index=False)


# 定义自动分箱函数
 
from scipy import stats
def mono_bin(Y, X, n = 20):
    r = 0
    good=Y.sum()
    bad=Y.count()-good
    while np.abs(r) < 1:
        d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n,duplicates="drop")}) 
        # 后面报错You can drop duplicate edges by setting the 'duplicates' kwarg，所以回到这里补充duplicates参数
        # pandas中使用qcut()，边界易出现重复值，如果为了删除重复值设置 duplicates=‘drop’，则易出现于分片个数少于指定个数的问题
        # 经尝试，设置duplicates参数为“drop”可行，而不能设置为“raise”。
        d2 = d1.groupby('Bucket', as_index = True)
        r, p = stats.spearmanr(d2.X.mean(), d2.mean().Y)
        n = n - 1
    d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
    d3['min'] = d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe']=np.log((d3['rate']/(1-d3['rate']))/(good/bad))
    d4 = (d3.sort_values(by='min')).reset_index(drop=True)
    print("=" * 60)
    woe=list(d4['woe'].round(3))
    print(d4)
    return d4

In [ ]:
#应变量
Y=train['SeriousDlqin2yrs']
# dict_Y = {'month':Y.index,'numbers':Y.values}
# df_month = pd.DataFrame(dict_Y)
#自变量，剔除对因变量影响不明显的变量
X=X_train.drop(['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate','NumberOfTime60-89DaysPastDueNotWorse','DebtRatio','MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans','NumberRealEstateLoansOrLines','NumberOfDependents'],axis=1)
X1=sm.add_constant(X)
Y1=Y.to_frame(name='SeriousDlqin2yrs')
logit=sm.Logit(Y,X1)
result=logit.fit()
print(result.summary())
X1.columns

如上图所示逻辑回归各变量都已通过显著性检验，满足要求

到这里，我们的建模部分基本结束了。我们需要验证一下模型的预测能力如何。我们使用在建模开始阶段预留的test数据进行检验。通过ROC曲线和AUC来评估模型的拟合能力。
在Python中，可以利用sklearn.metrics，它能方便比较两个分类器，自动计算ROC和AUC。

In [ ]:
#6.3 模型检验
 
X_test = trans_woe(X_test,x1_name,woex1,cutx1)
X_test = trans_woe(X_test,x2_name,woex2,cutx2)
X_test = trans_woe(X_test,x3_name,woex3,cutx3)
X_test = trans_woe(X_test,x7_name,woex7,cutx7)
X_test = trans_woe(X_test,x9_name,woex9,cutx9)
 
#应变量
Y_test = test['SeriousDlqin2yrs']
#自变量，剔除对因变量影响不明显的变量，与模型变量对应
X_test = X_test.iloc[:,-5:]
#X_test =X_test.drop(['NumberOfOpenCreditLinesAndLoans','age','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate','NumberOfTime60-89DaysPastDueNotWorse','DebtRatio','MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans','NumberRealEstateLoansOrLines','NumberOfDependents'], axis=1)
X3 = sm.add_constant(X_test)
resu = result.predict(X3)#进行预测
#result.score(X3,Y_test) 
fpr, tpr, threshold = metrics.roc_curve(Y_test, resu)
rocauc = metrics.auc(fpr, tpr)#计算AUC
plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % rocauc)#生成ROC曲线
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('真正率')
plt.xlabel('假正率')
plt.show()

信用评分
我们已经基本完成了建模相关的工作，并用ROC曲线验证了模型的预测能力。接下来的步骤，就是将Logistic模型转换为标准评分卡的形式。

 

评分标准：

a=log（p_good/P_bad）

Score = offset + factor * log(odds)

在建立标准评分卡之前，我们需要选取几个评分卡参数：基础分值、 PDO（比率翻倍的分值）和好坏比。 这里， 我们取600分为基础分值，PDO为20 （每高20分好坏比翻一倍），好坏比取20。

In [ ]:
# 我们取600分为基础分值，PDO为20（每高20分好坏比翻一倍），好坏比取20。
p = 20 / math.log(2)
q = 600 - 20 * math.log(20) / math.log(2)
#7.2 部分评分
#计算分数函数 
def get_score(coe,woe,factor):
    scores=[]
    for w in woe:
        score=round(coe*w*factor,0)
        scores.append(score)
    print(scores)    
    return scores
   
 
 
coe = [2.6138,0.6228,0.4894,0.5596,0.5747 ,0.4248]#在上面那个logistic的结果中来
baseScore = round(q + p * coe[0], 0)
print(baseScore)

In [ ]:

# 各项部分分数
x1 = get_score(coe[1], woex1, p)
x2 = get_score(coe[2], woex2, p)
x3 = get_score(coe[3], woex3, p)
x7 = get_score(coe[4], woex7, p)
x9 = get_score(coe[5], woex9, p)


自动评分系统
根据变量来计算分数，实现如下：

In [ ]:
def compute_score(series,cut,score):
    list = []
    i = 0
    while i < len(series):
        value = series[i]
        j = len(cut) - 2
        m = len(cut) - 2
        while j >= 0:
            if value >= cut[j]:
                j = -1
            else:
                j -= 1
                m -= 1
        list.append(score[m])
        i += 1
    return list


In [ ]:
#test = pd.read_csv('./TestData.csv')
test = pd.read_csv('../input/GiveMeSomeCredit/cs-test.csv')
test['BaseScore']=np.zeros(len(test))+baseScore
test['x1'] = compute_score(test['RevolvingUtilizationOfUnsecuredLines'], cutx1, x1)
test['x2'] = compute_score(test['age'], cutx2, x2)
test['x3'] = compute_score(test['NumberOfTime30-59DaysPastDueNotWorse'], cutx3, x3)
test['x7'] = compute_score(test['NumberOfTimes90DaysLate'], cutx7, x7)
test['x9'] = compute_score(test['NumberOfTime60-89DaysPastDueNotWorse'], cutx9, x9)
test['Score'] = test['x1'] + test['x2'] + test['x3'] + test['x7'] +test['x9']  + baseScore
test.to_csv('ScoreData.csv', index=False)

import math

df1 = test.loc[:,['Score']]
df1.sort_values(by='Score',axis=0,ascending=True,inplace=True)
df1['Score'][20000]
print(df1['Score'][101503-math.ceil(rr*101503)])
print(math.ceil(rr*101503))
print(df1['Score'][6701])
display(df1)
te=0
j=0
for te in df1['Score']:
    if(j >= rr*101503):
        break
    j=j+1
print(j)
print(te)

In [ ]:

test['Score'] = (test['Score'] - 622)/(745 - 620)
print(test['Score'][80000])

xx=0
for i in test['Score']:
    if i >= 0:
        test['Score'][xx] = 0
    else:
        test['Score'][xx] = 1
    xx=xx+1
    
        


tt = pd.read_csv('../input/GiveMeSomeCredit/cs-test.csv')
score = pd.DataFrame({
        "Id": tt["Unnamed: 0"],
        "Probability": test['Score']
    })

score.to_csv('score.csv', index=False)